In [1]:
from __future__ import division

import pathlib
%matplotlib inline


# ignore warning
import warnings
warnings.filterwarnings('ignore')

import os, cv2
import tensorflow as tf
import numpy as np
from networks import build_aggasatt_joint

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
# tf.config.experimental.set_visible_devices([], 'GPU')

In [3]:
vgg19_path = './Models/imagenet-vgg-verydeep-19.mat'
pretrain_model_path = './Models/istd-pretrained/'

# original = "./original"
# source = "./source/"
# target = "./target/"

original = "./test/"
source = "./source/"
target = "./target/"

channel = 32

In [4]:
with tf.variable_scope(tf.get_variable_scope()):
    input=tf.placeholder(tf.float32,shape=[None,None,None,3])
    shadow_free_image=build_aggasatt_joint(input,channel,vgg19_path)

sess=tf.Session()
sess.run(tf.global_variables_initializer())
idtd_ckpt=tf.train.get_checkpoint_state(pretrain_model_path)
saver_restore=tf.train.Saver([var for var in tf.trainable_variables()])
print('loaded '+ idtd_ckpt.model_checkpoint_path)
saver_restore.restore(sess,idtd_ckpt.model_checkpoint_path)




[i] Hypercolumn ON, building hypercolumn features ... 


Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Use keras.layers.AveragePooling2D instead.




loaded ./Models/istd-pretrained/lasted_model.ckpt
INFO:tensorflow:Restoring parameters from ./Models/istd-pretrained/lasted_model.ckpt


In [5]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (24,6)

imgs = [os.path.join(original,x) for x in os.listdir(original) if '.jpg' in x or '.png' in x]

print(imgs)

['./test/173.jpg']


In [6]:
height = 432
width = int(4 / 3 * height)

print(f"Width = {width}, Height = {height}")

for img_path in imgs:

    iminput = cv2.resize(cv2.imread(img_path, 1), (width, height))
    # iminput = cv2.resize(cv2.imread(img_path, 1), (1920, 1440))

    imoutput = sess.run(shadow_free_image,feed_dict={input:np.expand_dims(iminput/255.,axis=0)})
    imoutput = np.uint8(np.squeeze(np.minimum(np.maximum(imoutput[0],0.0),1.0))*255.0)

    imname = pathlib.Path(img_path).stem
    cv2.imwrite(os.path.join(source, f'{imname}.jpg'), iminput)
    cv2.imwrite(os.path.join(target, f'{imname}.jpg'), imoutput)
    cv2.waitKey(0)

Width = 580, Height = 435


ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted: OOM when allocating tensor with shape[1,435,580,1475] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node concat_4 (defined at E:\OpenSoftware\miniconda\envs\shadow_synthesis\lib\site-packages\tensorflow_core\python\framework\ops.py:1748) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted: OOM when allocating tensor with shape[1,435,580,1475] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node concat_4 (defined at E:\OpenSoftware\miniconda\envs\shadow_synthesis\lib\site-packages\tensorflow_core\python\framework\ops.py:1748) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[g_conv_img/BiasAdd/

In [ ]:
print("success")

In [ ]:
# images should be in 4:3 ratio
# checking possible resolutions

# height = 435
# width = 4 / 3 * height
#
# width

In [ ]:
lower = 432

upper = 435
# right now model works until height is 435
# my task is to make it work for height = 435 and up with little to no loss to quality

In [ ]:
# img_path = "E:\Semester 7 - Moodle 2021\FYP\projects\DHAN\Dataset\\NIC_Dataset\\train\\train_B\\1.jpg"
#
# import cv2
# import numpy as np
#
# ###test image
# img=cv2.imread(img_path)
#
# ### splitting b,g,r channels
# b,g,r=cv2.split(img)
#
# ### getting differences between (b,g), (r,g), (b,r) channel pixels
# r_g=np.count_nonzero(abs(r-g))
# r_b=np.count_nonzero(abs(r-b))
# g_b=np.count_nonzero(abs(g-b))
#
# ### sum of differences
# diff_sum=float(r_g+r_b+g_b)
#
# ### finding ratio of diff_sum with respect to size of image
# ratio=diff_sum/img.size
#
# if ratio>0.005:
#     print("image is color")
# else:
#     print("image is greyscale")
